# Introduction to numpy for image processing

In image processing we often want to manipulate images through programming. This is best done with high level programming languages such as python or Matlab. This course's assignments will mostly focus on image processing with python. 

Digital images are arrays of pixels in three dimensions *HxWxC*. *C* stands for channel or color channel and the size of that dimension is equal to the number of color channels. Three examples of common color spaces are RGB, HSV and CMYK.

## Introduction

### First look at numpy
Numpy is a python library for matrix manipulation and adds support for a number of high-level mathematical operations. Numpy allows for vectorization of operations which is alot faster than traditional for-loops in python.

To get started with numpy we must import the library. The short-hand np is standarized, and is the recommended way to import numpy.

In [ ]:
import numpy as np

We can then declare zero-initialized arrays or declare arrays from existing python lists. There are of course a number of other possible options for initializing our arrays which can be explored in the [docs](https://numpy.org/doc/stable/reference/routines.array-creation.html).

In [ ]:
python_list = [[1.  if (i + j) % 2 == 0 else 0. for i in range(8)] for j in range(8)]

np_from_list = np.array(python_list)
print(np_from_list)

#When initializing an array without any data we must provide the dimensions
np_zero = np.zeros((8,8))
print(np_zero)

Most often we do not want to view the matrices as numbers, we want to visualize them as images. This can be done by utilising another python library which builds on numpy: matplotlib.

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(np_from_list, cmap = "gray")
plt.show()
plt.imshow(np_zero, cmap = "gray")
plt.show()

### Importing images

We now demonstrated how to visualize images, but checkerboards and all black frames are not very interesting. This next step will show how we can utilise matplotlib to read images and import them into numpy arrays.

In [ ]:
image_path = "images/strawberry.jpg"

im_array = plt.imread(image_path)
plt.imshow(im_array)
plt.show()

Often we want to know the dimensions of our image. This can be done like this:

In [ ]:
shape = im_array.shape
print(f"Image dimensions: {shape}")

From this we see that pyplot imports images in the order *HxWxC*.

### Intensity transformations

Some of the simplest image processing operations are intensity transformations or point processing operations. Intensity transformations are image operations that operate on each pixel individually without respect to the neighbouring pixels. This is in contrast to neighbourhood processing, which assesses the neighbourhood around each pixel in order to transform it.

A simpel operation is inverting the image. With numpy this can be done easily, as all matematical operations are implicitly done on all elements of the array.

In [ ]:
inverted = 255 - im_array
plt.imshow(inverted)
plt.show()

In intensity transforms each pixel is assigned a new color according to a scalar function:
$$c' = f(c)$$

The function f can vary alot depending on what we want to do. The following snippet displays some common mathematical operations that we can apply to images.

In [ ]:
#First we want to convert our image from an [0, 256] uint8 array to a [0.0, 1.0] float array.
float_im = im_array/255.

log_transform = np.log(1 + float_im)
plt.imshow(log_transform)
plt.show()

exp_transform = (np.exp(float_im) - 1)/(np.e - 1)
plt.imshow(exp_transform)
plt.show()

gamma_transform = np.power(float_im, 0.5)
plt.imshow(gamma_transform)
plt.show()

gamma_transform = np.power(float_im, 3)
plt.imshow(gamma_transform)
plt.show()


threshold_transform = np.where(float_im > 0.5, 1.0, 0.0)
plt.imshow(threshold_transform)
plt.show()

### Slicing in numpy

Numpy arrays can be indexed and sliced as normal python lists, but the syntax is a bit different.

In [ ]:
im_copy = im_array.copy()
im_copy[100:400, 100:400, :] = [0, 0, 255]
plt.imshow(im_copy)
plt.show()

### Histogram equalization

Histogram equalization is a method used in image processing to improve contrast in images. It stretches the dynamic range for the most frequent intensity values, effectively spreading them out over the whole range of possible values. Normally this will imporve the contrast of images, brighten dark images and darken bright images.

In [ ]:
def histogram_eq(image):
    #First we calculate the histogram.
    hist, bins = np.histogram(image[:,:], 256)
    
    #Then we calculate the cumulative histogram
    cumulative_hist = np.zeros((256), dtype = np.int)
    cumulative_hist[0] = hist[0]
    for i in range(1, 256):
        cumulative_hist[i] = cumulative_hist[i-1] + hist[i]
    
    #Then we normalize the histogram to [0., 1.]
    normalized_cumulative_hist = cumulative_hist.astype(float)
    normalized_cumulative_hist /= normalized_cumulative_hist[-1]
    
    #Then we construct an intensity mapping table by multiplying with the maximum intensity value
    table = (normalized_cumulative_hist * 255).astype(np.uint8)
    
    #Lastly we create the new image by mapping each of the old intensities to the new corresponding intensity
    new_im = table[image[:,:]]
    return new_im


image = plt.imread("images/low_contrast_butterfly.tiff")[:,:,0]
plt.figure(figsize = (20,5))
plt.subplot(1,2,1)
plt.imshow(image, cmap = "gray")
image = histogram_eq(image)
plt.subplot(1,2,2)
plt.imshow(image, cmap = "gray")
plt.show()



image = plt.imread("images/high_contrast_butterfly.tiff")[:,:,0]
plt.figure(figsize = (20,5))
plt.subplot(1,2,1)
plt.imshow(image, cmap = "gray")
image = histogram_eq(image)
plt.subplot(1,2,2)
plt.imshow(image, cmap = "gray")
plt.show()




## Exercises

### 1. Array creation
Declare a 255x255x1 black/white image which is all black except for a white circle in the middle. Display the image to screen.

In [ ]:
#Write your code here

#### Example solution

In [ ]:
python_list = [[1. if ((i-127)**2 + (j-127)**2 < 900) else 0. for i in range(255)] for j in range(255)]
np_array = np.array(python_list)
plt.imshow(np_array, cmap = "gray")
plt.show()

### 2. Intensity transform

Import the image from the given path. Apply a gamma transform to a rectangular slice of the image.

In [ ]:
im_path = "images/eye_closeup.jpg"
#Write your code here.

#### Example solution

In [ ]:
image = plt.imread(im_path)
image = image/255.

image[100:800, 100:750, :] = np.power(image[100:800, 100:750, :], 3)

plt.imshow(image)
plt.show()

### 3. Creative task

Take the original strawberry image from the introduction and see if you can change the background color from blue to something else without affecting the non-background objects too much.

In [ ]:
image_path = "images/strawberry.jpg"

image = plt.imread(image_path)

#### Example solution
This could probably be vectorized for performance, but displays the idea nicely.

In [ ]:
result = np.zeros_like(image)
for y in range(image.shape[0]):
    for x in range(image.shape[1]):
        #Set the pixel to black if blue is the dominant color
        result[y,x,:] = image[y,x,:] if image[y,x,0]**2 + image[y,x,1]**2 > image[y,x,2]**2 else [0,0,0]
plt.imshow(result)
plt.show()

result = np.zeros_like(image)
for y in range(image.shape[0]):
    for x in range(image.shape[1]):
        result[y,x,:] = image[y,x,:] if image[y,x,0]**2 + image[y,x,1]**2 < image[y,x,2]**2 else [0,0,0]
plt.imshow(result)
plt.show()